## Importing modules/libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

from scipy.stats import norm 
from scipy.special import gamma 

# for interactive plots
%matplotlib notebook

# Structural Reliability Workshop 2022

University of Liège, Belgium.

Moderators: Felipe Giro, Jose Mishael, Pablo G. Morato.

## Outline 
- Monte carlo overview (10 mins).
- Fatigue damage estimation overview (10 mins).
- Applied exercise (20 mins).
- Q&A (5 mins).

# Monte Carlo (sampling) method

**Objective:** Compute the exceeding probability of an event defined as a function of a random variable $X$. Then compare your result with the analytical solution.

In [ ]:
# random variable described by a Gaussian distribution with parameters:
mean, std = 4, 1.2

# exceeding probability defined based on the following threshold:
threshold = 5

In [ ]:
# analytical solution:
Analytical_sol = norm(mean, std).sf(threshold)

print("Analytical solution: the exceeding probability equals {:.4f}".format(Analytical_sol))

In [ ]:
# Monte Carlo (sampling) integration

number_samples = int(1e6)

# samples generation
samples = np.random.normal(mean, std, size=number_samples)

# counting all samples above the predefined threshold
n_samples_in = (samples > threshold).sum()

MonteCarlo_solution = n_samples_in/number_samples 

print("Monte Carlo solution: the exceeding probability equals {:.4f}".format(MonteCarlo_solution))

In [ ]:
# for plotting purposes
x_axis = np.linspace(-1, 9)
fig, ax = plt.subplots()
ax.axvline(threshold, color="m", label="threshold", ls="--")
ax.plot(x_axis, norm(mean, std).pdf(x_axis), lw=4, alpha=.8, label='Analytical')
ax.hist(samples, bins=50, density=True, color="green", alpha=.5, label="Sampling (Monte Carlo)")
ax.set_xlabel('x')
ax.set_ylabel('pdf')
ax.legend()
ax.axvspan(threshold, 9, color="grey", alpha=.25)
ax.set_xlim((-1, 9))
plt.show()

# Why Monte Carlo sampling is useful?

- Intractable integration in most practical cases.
- Curse of dimensionality: when treating more than 5-6 random variables.

Let's consider the previous example, yet this time, the exceeding probability corresponds to an event described by a non-linear function: $f(x)=1/3\cdot x^{2}$.

In [ ]:
f = lambda x : 1/3*x**2

samples1 = f(samples)

In [ ]:
# Calculate pf based on the predefined threshold
pf_x0 = MonteCarlo_solution # already computed
pf_x1 = (samples1 > threshold).sum()/number_samples

In [ ]:
# for plotting purposes
fig, ax = plt.subplots()

ax.hist(samples , bins=100, alpha=.6, density=True, label="$x_0, Pr[x>{}]={:.3f}$".format(threshold, pf_x0))
ax.hist(samples1, bins=100, alpha=.6, density=True, label="$x_1=f(x_0), Pr[x>{}]={:.3f}$".format(threshold, pf_x1))
ax.legend()
ax.set_xlim((-1,15))
ax.set_xlabel('x')
ax.set_ylabel('Probability mass function (pmf)')
plt.show()

# Exercise: fatigue damage estimation

Compute the failure probability of a structural element via Miner's rule.

#### Damage calculation 
(Equation 5.1.3, DNVGL-RP-C203)

$D=\frac{v_0 T_d}{a}q^m\Gamma(1+\frac{m}{h})$    

#### Limit state function

$g = \Delta - D $ 

#### Random variables

| Variable | Distribution | Description |
| --- | --- | --- |
| $a$ | lognormal | intercept of the design S-N curve with the log N axis |
| $q$ | normal | Weibull stress range scale distribution parameter |
| $\Delta$ |lognormal | inital damage state |

In [ ]:
# number of samples
n_samples = int(1e6)

# parameters (constant)
v0 = 5049216 # cycles
m = 3 # negative inverse slope of the S-N curve
h = 0.8 # shape parameters for the Weibull stress range

# parameters (random variables)
loga = 11.687
loga_std = 0.2
loga_mean = loga + 2*loga_std
q_mean = 4.59
q_std = 0.25*q_mean
Delta_mean = 0 # in numpy, it use the corresponding normal parameters
Delta_std = (np.log(0.3**2+1))**0.5 # in numpy, it uses the corresponding normal parameters

# random variables
loga = ### YOUR CODE HERE ###
Delta = ### YOUR CODE HERE ###
q = ### YOUR CODE HERE ###

In [ ]:
# for plotting purposes
fig, axes = plt.subplots(ncols=3, figsize=(8,3))

fig.suptitle("Fatigue damage computation random variables")

axes[0].hist(loga, bins=100, density=True)
axes[0].set_xlabel("$log_{10}a$")
axes[0].set_ylabel("pmf")

axes[1].hist(Delta, bins=100, density=True)
axes[1].set_xlabel("$\Delta$")
axes[1].set_ylabel("pmf")

axes[2].hist(q, bins=100, density=True)
axes[2].set_xlabel("$q$")
axes[2].set_ylabel("pmf")

plt.tight_layout()
plt.show()

In [ ]:
# damage propagation after 20 years
Td = 20

# Deterioration equation
### YOUR CODE HERE ### (if necessary)

a = 10**loga
g = ### YOUR CODE HERE ###
pf = ### YOUR CODE HERE ###

print("Probability of failure:", pf)

In [ ]:
# for plotting purposes
fig, ax = plt.subplots(figsize=(8,3))

fig.suptitle("Deterioration after {} years".format(Td))

ax.hist(g, density=True, bins=100, range=(-0.5, 2.5))
ax.set_xlabel("Damage limit state")
ax.set_ylabel("pmf")
ax.axvline(0, color='m', ls="--")

plt.tight_layout()
plt.show()

## Deterioration state evolution

In [ ]:
# Initial deterioration state
g_0  = ### YOUR CODE HERE ###
pf_0 = ### YOUR CODE HERE ###

# Deterioration state after 10 years
g_10 = ### YOUR CODE HERE ###
pf_10 = ### YOUR CODE HERE ###

# Deterioration state after 20 years
g_20 = ### YOUR CODE HERE ###
pf_20 = ### YOUR CODE HERE ###

In [ ]:
# for plotting purposes
fig, ax = plt.subplots(figsize=(8,3))

fig.suptitle("Deterioration evolution for {} year lifetime".format(Td))

ax.set_xlim((-0.2, 2.5))

ax.hist(g_0, bins=100, density=True, histtype='step', color = '0.8', label="Year 0, Pf={:.4f}".format(pf_0*100), lw=1.5, range=(-0.2, 2.5))
ax.hist(g_10, bins=100, density=True, histtype='step', color = '0.5', label="Year 10, Pf={:.4f}".format(pf_10*100), lw=1.5, range=(-0.2, 2.5))
ax.hist(g_20, bins=100, density=True, histtype='step', color = 'k', label="Year 20, Pf={}".format(pf_20*100), lw=1.5, range=(-0.2, 2.5))
ax.axvline(0, color='m', ls="--")

ax.set_xlabel("")
ax.legend()

plt.show()

## Sensitivity on the assigned uncertainties

Try to change the random variables' uncertainties (represented, in this case, by the standard deviation). What is the influence of parameters uncertainties on the probability of failure ($p_f$)? By reducing the uncertainties, are we able to reduce the probability of failure? Is it always the case?

In [ ]:
# parameters (random variables)
loga_std = 0.2
loga_mean = loga + 2*loga_std
q_mean = 4.59
q_std = 0.25*q_mean
Delta_mean = 0 # in numpy, it use the corresponding normal parameters
Delta_std = (np.log(0.3**2+1))**0.5 # in numpy, it uses the corresponding normal parameters

# random variables
loga = ### YOUR CODE HERE ###
Delta = ### YOUR CODE HERE ###
q = ### YOUR CODE HERE ###

g_user = ### YOUR CODE HERE ###
pf_user = ### YOUR CODE HERE ###

# for plotting proposes
fig, ax = plt.subplots(figsize=(8,3))
ax.hist(g_20, bins=100, density=True, histtype='step', color = 'k', label="Original, pf={:.4f}%".format(pf_20*100), ls="-", lw=1.5, range=(-0.2, 2.5))
ax.hist(g_user, bins=100, density=True, histtype='step', color = 'red', label="User change, pf={:.4f}%".format(pf_user*100), ls="-", lw=1.5, range=(-0.2, 2.5))
ax.legend()
ax.axvline(0, color='m', ls="--")
plt.show()

# Discussion
- Why the values of $p_f$ are different?